In [1]:
#For Colab ENV
from google.colab import drive 
drive.mount('/gdrive')
!git clone https://github.com/FacerAin/Kor_Eng_NMT_System.git '/gdrive/My Drive/Colab Notebooks/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
Cloning into '/gdrive/My Drive/Colab Notebooks'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 18 (delta 2), reused 8 (delta 0), pack-reused 0
Unpacking objects: 100% (18/18), done.


## NMT_seq2seq_basic

Character Level NMT based on Simple Seq2Seq Model Without Attention Mechanism 

Refer to https://github.com/keras-team/keras/blob/master/examples/lstm_seq2seq.py

In [27]:
import numpy as np 
from keras.models import Model
from keras.layers import Input, LSTM, Dense

In [34]:
# Parameter Setting
dataset_dir_path = '../dataset/kor-eng/'
batch_size = 64
latent_dim = 256
epochs = 100

In [21]:
# Text data
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [22]:
# Read Dataset and Preprocessing
with open(dataset_dir_path+"kor.txt", "r", encoding='utf-8') as f:
    lines = f.read().split('\n')
    for line in lines[:len(lines)-2]:#To Except Last Line
        split_line = line.split('\t')
        input_text = split_line[0]
        target_text = '^' + split_line[1]+ '&' #Using '^' to start tag, '&' to end tag
        input_texts.append(input_text)
        target_texts.append(target_text)
        for char_item in input_text:
            input_characters.add(char_item)
        for char_item in target_text:
            target_characters.add(char_item)

In [23]:
input_characters = sorted(input_characters)
target_characters = sorted(target_characters)

encoder_token_num = len(input_characters)
target_token_num = len(target_characters)

max_encoder_seq_length = max([len(item) for item in input_texts])
max_decoder_seq_length = max([len(item) for item in target_texts])

encoder_token_index_dict = dict([(char_item, i) for i, char_item in enumerate(input_characters)])
target_token_index_dict = dict([(char_item, i) for i, char_item in enumerate(target_characters)])

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, encoder_token_num), dtype='float32')
decoder_input_data = np.zeros((len(target_texts), max_decoder_seq_length, target_token_num), dtype='float32')
decoder_target_data = np.zeros((len(target_texts), max_decoder_seq_length, target_token_num), dtype='float32')

In [24]:
for i,(input_text, target_text) in enumerate(zip(input_texts,target_texts)):
    for j, char_item in enumerate(input_text):
        encoder_input_data[i, j, encoder_token_index_dict[char_item]] = 1.
    encoder_input_data[i, j+1:, encoder_token_index_dict[" "]] = 1.
    for j, char_item in enumerate(target_text):
        decoder_input_data[i, j, target_token_index_dict[char_item]] = 1.
        if j > 0:
            decoder_target_data[i , j-1, target_token_index_dict[char_item]] = 1.
        decoder_input_data[i, j+1: ,target_token_index_dict[" "]] = 1.
        decoder_target_data[i, j:, target_token_index_dict[" "]] = 1.

In [29]:
# Define the Encoder
encoder_inputs = Input(shape = (None, encoder_token_num))
encoder_lstm = LSTM(latent_dim, return_state = True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

In [30]:
#Define the Decoder
decoder_inputs = Input(shape = (None, target_token_num))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(target_token_num, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [31]:
#Define the Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [35]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
model.save('s2s.h5')   

Epoch 1/100


KeyboardInterrupt: 